In [21]:
"""
Part 1: Fetch Data
Reads the catalog and gets Target, RA, Dec, Date Observed, Time
Observed, and path tot he corresponding FITS file
"""

import os
import csv
import glob

# Paths
catalog = "epoch2_continuum_catalog.csv"
fits_dir = "epoch2_continuum_fits"

# Arrays for data
target = []
ra_deg = []
dec_deg = []
date = []
time = []
fits_path = []

# Read catalog and parse data
with open(catalog, newline = '') as f:
    reader = csv.DictReader(f)
    for row in reader:
        tgt = row.get('Target', '')
        d = row.get('Date Start', '')
        t = row.get('Median Time', '')
        try:
            ra = float(row.get('RA', ''))
            ra = round(ra, 5)
            dec = float(row.get('Dec', ''))
            dec = round(dec, 5)
        except (TypeError, ValueError):
            ra = None
            dec = None

        # Get corresponding filepath
        pattern = os.path.join(fits_dir, f"*{tgt}*.tt0.subim.fits")
        matches = glob.glob(pattern)
        filepath = matches[0] if matches else None

        # Append values to arrays
        target.append(tgt)
        ra_deg.append(ra)
        dec_deg.append(dec)
        date.append(d)
        time.append(t)
        fits_path.append(filepath)

In [ ]:
"""
Part 2: Generate Plots
Opens the filepath of each target, generates cutouts for the FITS, Legacy
Survey (DR10), and unWISE (NEO7) coordinates, find the Max Flux in mJy in
the cutout pixels, and displays the data in a plot
"""

import warnings
import pandas as pd
import numpy as np
from astropy.io import fits
from astropy.wcs import WCS
import matplotlib.pyplot as plt
import requests
from io import BytesIO
from PIL import Image
from matplotlib.patches import Circle
from astropy.coordinates import SkyCoord
from scipy.ndimage import gaussian_filter

# Paths
outdir = "continuum_plots"
os.makedirs(outdir, exist_ok=True)

# Append max flux to catalog
def append_flux(cat_df, target, max_flux_mjy):
    col_name = "Max Flux (mJy)"
    if col_name not in cat_df.columns:
        cat_df[col_name] = np.nan

    mask = cat_df["Target"] == target
    if mask.sum() == 0:
        warnings.warn(f"[SKIP] Flux append - {target} not found")
        return cat_df

    if pd.isna(cat_df.loc[mask, col_name]).all():
        cat_df.loc[mask, col_name] = max_flux_mjy
    return cat_df

# Legacy cutout
def get_legacy_sky_image(ra, dec, size=256):
    url = f"https://www.legacysurvey.org/viewer/jpeg-cutout?ra={ra}&dec={dec}&layer=ls-dr10&size={size}"
    r = requests.get(url, timeout=30)
    r.raise_for_status()
    return Image.open(BytesIO(r.content))

# unWISE cutout
def get_unwise_image(ra, dec, size=256):
    url = f"https://www.legacysurvey.org/viewer/jpeg-cutout?ra={ra}&dec={dec}&layer=unwise-neo7&size={size}"
    r = requests.get(url, timeout=30)
    r.raise_for_status()
    return Image.open(BytesIO(r.content))

# Extract 2D WCS
def read_continuum_fits(fits_path):
    hdul = fits.open(fits_path)
    data = hdul[0].data
    header = hdul[0].header
    if "CTYPE1" in header and "CTYPE2" in header:
        wcs = WCS(header)
        try:
            wcs = wcs.celestial
        except Exception:
            pass
    else:
        wcs = None
    hdul.close()

    # Squeeze dimensions
    if data is not None and data.ndim > 2:
        data = np.squeeze(data)

    return data, header, wcs

# Generate plot
def generate_cutout():
    try:
        df = pd.read_csv(catalog)
    except Exception as e:
        raise FileNotFoundError(f"[ERROR] Could not read {catalog}: {e}")

    # Ensure columns exist
    cols = {"Target", "RA", "Dec"}
    if not cols.issubset(df.columns):
        missing = cols - set(df.columns)
        raise KeyError(f"[ERROR] {catalog} missing: {missing}")
    if "Date Start" not in df.columns:
        df["Date Start"] = ""
    if "Median Time" not in df.columns:
        df["Median Time"] = ""

    # Get column data
    for idx, row in df.iterrows():
        target = str(row["Target"])
        ra = float(row["RA"])
        dec = float(row["Dec"])
        date_obs = row["Date Start"]
        time_obs = row["Median Time"]

        # Create Max Flux column
        if "Max Flux (mJy)" in df.columns and not pd.isna(row.get("Max Flux (mJy)")):
            continue

        # Find FITS file
        pattern = os.path.join(fits_dir, f"*{target}*.fits")
        matches = glob.glob(pattern)
        if not matches:
            warnings.warn(f"[SKIP] No FITS for {target}")
            continue
        fits_path = matches[0]

        try:
            data, header, wcs = read_continuum_fits(fits_path)
        except Exception as e:
            warnings.warn(f"[ERROR] Couldn't read {target}: {e}")
            continue

        max_pixel = np.nanmax(data)
        max_flux_mjy = round(max_pixel * 1000.0, 5)
        df = append_flux(df, target, max_flux_mjy)

        # Layout
        fig = plt.figure(figsize=(14, 4))
        suptitle = f"{target}\nDate: {date_obs} / Time: {time_obs}\nRA: {ra:.5f} (deg) / Dec: {dec:.5f} (deg)"
        fig.suptitle(suptitle, fontsize=12, y=0.95)

        # Continuum cutout
        if (wcs is not None) and (not np.any(np.isnan(data))):
            try:
                center_coord = SkyCoord(ra, dec, unit="deg")
                x_center, y_center = wcs.world_to_pixel(center_coord)
                x_center = int(np.round(x_center))
                y_center = int(np.round(y_center))

                # Cutout zoom
                half_size = 12
                ny, nx = data.shape

                # Bounds
                x_min = max(x_center - half_size, 0)
                x_max = min(x_center + half_size, nx)
                y_min = max(y_center - half_size, 0)
                y_max = min(y_center + half_size, ny)

                # Pad edges with NaN if on edge
                cutout = data[y_min:y_max, x_min:x_max]
                actual_ny, actual_nx = cutout.shape

                pad_x1 = max(0, half_size - x_center)
                pad_y1 = max(0, half_size - y_center)
                pad_x2 = max(0, (x_center + half_size) - nx)
                pad_y2 = max(0, (y_center + half_size) - ny)

                cutout = np.pad(cutout, ((pad_y1, pad_y2), (pad_x1, pad_x2)), mode="constant", constant_values=np.nan)

            except Exception:
                # Fallback
                cutout = data
        else:
            cutout = data

        ax1 = fig.add_subplot(1, 3, 1)
        vmin = np.nanmin(cutout)
        vmax = np.nanmax(cutout)
        ax1.imshow(cutout, origin="lower", cmap="jet", vmin=vmin, vmax=vmax)
        ax1.axis("off")
        ax1.set_title(f"Continuum Cutout\nMax Flux (mJy): {max_flux_mjy:.5f}", fontsize=12)

        # Contour lines (smoothed to 1 sigma)
        cutout_smooth = gaussian_filter(cutout, sigma=1)
        levels = np.linspace(vmin, vmax, 8)
        ax1.contour(cutout_smooth, levels=levels, colors="white", linewidths=1.5, origin="lower")

        # Legacy cutout
        ax2 = fig.add_subplot(1, 3, 2)
        leg_img = get_legacy_sky_image(ra, dec, size=256)
        ax2.imshow(np.array(leg_img), origin="lower", cmap="gray")
        ax2.axis("off")
        ny2, nx2 = np.array(leg_img).shape[:2]
        circ = Circle((nx2 / 2, ny2 / 2), radius=10, edgecolor="blue", facecolor="none", linewidth=1)
        ax2.add_patch(circ)
        ax2.set_title("Legacy Cutout", fontsize=12)

        # unWISE cutout
        ax3 = fig.add_subplot(1, 3, 3)
        uw_img = get_unwise_image(ra, dec, size=256)
        ax3.imshow(np.array(uw_img), origin="lower", cmap="gray")
        ax3.axis("off")
        ny3, nx3 = np.array(uw_img).shape[:2]
        circ2 = Circle((nx3 / 2, ny3 / 2), radius=10, edgecolor="blue", facecolor="none", linewidth=1)
        ax3.add_patch(circ2)
        ax3.set_title("unWISE Cutout", fontsize=12)

        plt.tight_layout(pad=0.5, rect=[0, 0, 1, 0.75])
        plt.subplots_adjust(wspace=-0.6, left=0.02, right=0.98, top=0.70)
        out_path = os.path.join(outdir, f"{target}_combined.png")
        plt.savefig(out_path, dpi=150)
        plt.show()
        plt.close()

    # Overwrite Max Flux column
    df.to_csv(catalog, index=False)
    print(f"[COMPLERE] Updated {catalog}")

if __name__ == "__main__":
    generate_cutout()